In [29]:
import os
import FinanceDataReader as fdr
from tqdm import tqdm
import pandas as pd

# Pycaret 이용

## 모든 종목 예측

* 종가와 상관계수 높은 특성 : KOSDAQ(0.87), KOSPI(0.79), DOW(0.58), NASDAQ(0.50), S&P500(0.56)
* 예측 순서(종가와 상관계수 높은 특성 먼저 예측)
    * weekday, weeknumdm로 KOSDAQ 예측(종가와 상관계수 0.87)
    * weekday, weeknum, KOSDAQ으로 NASDAQ 예측(KOSDAQ과 상관계수 0.68)
    * weekday, weeknum, KOSDAQ, NASDAQ으로 S&P500 예측(NASDAQ과 상관계수 0.93)
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500으로 DOW 예측(S&P500과 상관계수 0.95)
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500, DOW로 KOSPI 예측
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500, DOW, KOSPI로 다른 종가 예측 

In [30]:
# 제출 점수 :

In [31]:
from pycaret.regression import *

# 데이터 불러오기

## 외부 데이터 가져오기

In [32]:
# 외부 데이터 가져오기
df = pd.read_csv('../data/20210104_20211119_data_interpolate.csv')
df.head()

,Date,exchange_rate,kospi,kosdaq,kospi100,dow,nasdaq,sp500,미국채10년-2년,BTC,Gold,Oil,US10Y,US2Y
0,2021-01-04,1086.48,2944.45,977.62,3052.74,30223.89,12698.4,3700.65,0.8009,36499000.0,37.10,32.38,0.918,0.1171
1,2021-01-05,1086.42,2990.57,985.76,3099.42,30391.60,12819.0,3726.86,0.8320,37962000.0,37.18,33.94,0.955,0.1230
2,2021-01-06,1087.93,2968.21,981.39,3070.05,30829.40,12740.8,3748.14,0.9002,40731000.0,36.60,34.11,1.039,0.1388
3,2021-01-07,1094.28,3031.68,988.86,3139.48,31041.13,13067.5,3803.79,0.9402,44575000.0,36.48,34.53,1.081,0.1408
4,2021-01-08,1092.93,3152.18,987.79,3293.96,31097.97,13202.0,3824.68,0.9821,47241000.0,35.26,35.43,1.119,0.1369


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           230 non-null    object 
 1   exchange_rate  230 non-null    float64
 2   kospi          230 non-null    float64
 3   kosdaq         230 non-null    float64
 4   kospi100       230 non-null    float64
 5   dow            230 non-null    float64
 6   nasdaq         230 non-null    float64
 7   sp500          230 non-null    float64
 8   미국채10년-2년      230 non-null    float64
 9   BTC            230 non-null    float64
 10  Gold           230 non-null    float64
 11  Oil            230 non-null    float64
 12  US10Y          230 non-null    float64
 13  US2Y           230 non-null    float64
dtypes: float64(13), object(1)
memory usage: 25.3+ KB


In [34]:
df.columns

Index(['Date', 'exchange_rate', 'kospi', 'kosdaq', 'kospi100', 'dow', 'nasdaq',
       'sp500', '미국채10년-2년', 'BTC', 'Gold', 'Oil', 'US10Y', 'US2Y'],
      dtype='object')

### 필요한 특성만 추출

In [35]:
df = df[['Date','kosdaq','nasdaq','sp500','dow','kospi']]
df.head()

,Date,kosdaq,nasdaq,sp500,dow,kospi
0,2021-01-04,977.62,12698.4,3700.65,30223.89,2944.45
1,2021-01-05,985.76,12819.0,3726.86,30391.60,2990.57
2,2021-01-06,981.39,12740.8,3748.14,30829.40,2968.21
3,2021-01-07,988.86,13067.5,3803.79,31041.13,3031.68
4,2021-01-08,987.79,13202.0,3824.68,31097.97,3152.18


In [36]:
# 요일 번호와 주 번호 지정하여 열로 추가하기
df['Date'] = pd.to_datetime(df['Date'])
df['weekday'] = df['Date'].apply(lambda x : x.weekday())
df['weeknum'] = df['Date'].apply(lambda x : x.strftime('%V'))

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     230 non-null    datetime64[ns]
 1   kosdaq   230 non-null    float64       
 2   nasdaq   230 non-null    float64       
 3   sp500    230 non-null    float64       
 4   dow      230 non-null    float64       
 5   kospi    230 non-null    float64       
 6   weekday  230 non-null    int64         
 7   weeknum  230 non-null    object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 14.5+ KB


In [38]:
df.columns

Index(['Date', 'kosdaq', 'nasdaq', 'sp500', 'dow', 'kospi', 'weekday',
       'weeknum'],
      dtype='object')

In [39]:
df = df[['Date', 'weekday', 'weeknum','kosdaq', 'nasdaq', 'sp500', 'dow', 'kospi']]
df.head()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
0,2021-01-04,0,01,977.62,12698.4,3700.65,30223.89,2944.45
1,2021-01-05,1,01,985.76,12819.0,3726.86,30391.60,2990.57
2,2021-01-06,2,01,981.39,12740.8,3748.14,30829.40,2968.21
3,2021-01-07,3,01,988.86,13067.5,3803.79,31041.13,3031.68
4,2021-01-08,4,01,987.79,13202.0,3824.68,31097.97,3152.18


### 훈련, 테스트 셋 나누기

In [72]:
start_date = '2021-01-04'
end_date = '2021-10-29'

In [73]:
train = df[(df['Date'] <= end_date) & (df['Date'] >= start_date)]
test =  df[(df['Date'] <= '2021-11-05') & (df['Date'] >= '2021-11-01')]

In [74]:
train.shape, test.shape

((215, 8), (5, 8))

In [75]:
train

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
0,2021-01-04,0,01,977.62,12698.4,3700.65,30223.89,2944.45
1,2021-01-05,1,01,985.76,12819.0,3726.86,30391.60,2990.57
2,2021-01-06,2,01,981.39,12740.8,3748.14,30829.40,2968.21
3,2021-01-07,3,01,988.86,13067.5,3803.79,31041.13,3031.68
4,2021-01-08,4,01,987.79,13202.0,3824.68,31097.97,3152.18
...,...,...,...,...,...,...,...,...
210,2021-10-25,0,43,994.31,15226.7,4566.48,35743.78,3020.54
211,2021-10-26,1,43,1011.76,15235.7,4574.79,35755.83,3049.08
212,2021-10-27,2,43,1008.95,15235.8,4551.68,35491.48,3025.49
213,2021-10-28,3,43,1000.13,15448.1,4596.42,35729.89,3009.55


In [76]:
test.head()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
215,2021-11-01,0,44,998.57,15595.9,4613.67,35913.68,2978.94
216,2021-11-02,1,44,1009.44,15649.6,4630.65,36053.09,3013.49
217,2021-11-03,2,44,1005.00,15811.6,4660.57,36157.02,2975.71
218,2021-11-04,3,44,1001.43,15940.3,4680.06,36124.66,2983.22
219,2021-11-05,4,44,1001.35,15971.6,4697.53,36329.07,2969.27


In [77]:
test = test[['Date', 'weekday', 'weeknum']]
test

,Date,weekday,weeknum
215,2021-11-01,0,44
216,2021-11-02,1,44
217,2021-11-03,2,44
218,2021-11-04,3,44
219,2021-11-05,4,44


## 종목 정보 가져오기

In [78]:
# 종목 정보 가져오기
path = '../data'
list_name = 'Stock_List.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list.head()

,종목명,종목코드,상장시장
0,삼성전자,5930,KOSPI
1,SK하이닉스,660,KOSPI
2,NAVER,35420,KOSPI
3,카카오,35720,KOSPI
4,삼성바이오로직스,207940,KOSPI


In [79]:
stock_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목명     370 non-null    object
 1   종목코드    370 non-null    int64 
 2   상장시장    370 non-null    object
dtypes: int64(1), object(2)
memory usage: 8.8+ KB


In [80]:
# 종목 코드 6자리로 맞추기
stock_list['종목코드'] = stock_list["종목코드"].astype(str).str.zfill(6)
stock_list.head()

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI


# 종가 예측

* 1. kosdaq 예측
* 2. nasdaq
* 3. sp500
* 4. dow
* 5. kospi
* 6. 종가

In [81]:
train.head()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
0,2021-01-04,0,01,977.62,12698.4,3700.65,30223.89,2944.45
1,2021-01-05,1,01,985.76,12819.0,3726.86,30391.60,2990.57
2,2021-01-06,2,01,981.39,12740.8,3748.14,30829.40,2968.21
3,2021-01-07,3,01,988.86,13067.5,3803.79,31041.13,3031.68
4,2021-01-08,4,01,987.79,13202.0,3824.68,31097.97,3152.18


In [82]:
test

,Date,weekday,weeknum
215,2021-11-01,0,44
216,2021-11-02,1,44
217,2021-11-03,2,44
218,2021-11-04,3,44
219,2021-11-05,4,44


### 테스트셋의 5가지 특성 예측 하기

In [83]:
features = ['Date', 'weekday', 'weeknum','kosdaq', 'nasdaq', 'sp500', 'dow', 'kospi']
for i in range(5):
    model = setup(train, target = features[i+3], ignore_features = features[i+4:], fold=3, fold_shuffle=True, silent=True)
    tuned_model = tune_model(compare_models(sort='MAPE', n_select=1))
    final_model = finalize_model(tuned_model)
    end_features = features[:i+3]
    pred = predict_model(final_model, test)
    test[features[i+3]] = pred['Label']

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,16.5338,421.3691,20.5273,0.9357,0.0065,0.0052
1,24.6391,1052.7833,32.4466,0.8658,0.0105,0.0080
2,23.9168,1065.3026,32.6390,0.8799,0.0105,0.0077
Mean,21.6966,846.4850,28.5376,0.8938,0.0092,0.0070
SD,3.6625,300.6458,5.6647,0.0302,0.0019,0.0012


In [85]:
test

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
215,2021-11-01,0,44,1006.418941,14737.349456,4407.581144,33977.187500,3062.021995
216,2021-11-02,1,44,1007.549998,14770.291740,4428.330825,34165.136719,3064.841639
217,2021-11-03,2,44,1006.637302,14768.971554,4428.732729,34169.476562,3062.021995
218,2021-11-04,3,44,1007.108948,14793.617235,4452.543789,34432.042969,3059.876766
219,2021-11-05,4,44,1007.500128,14834.681366,4457.262093,34424.101562,3058.086760


In [84]:
test.to_csv('20210104_1029_pred_1101_05_test.csv', index=False)

In [86]:
test = pd.read_csv('20210104_1029_pred_1101_05_test.csv')
test

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
0,2021-11-01,0,44,1006.418941,14737.349456,4407.581144,33977.188,3062.021995
1,2021-11-02,1,44,1007.549998,14770.291740,4428.330825,34165.137,3064.841639
2,2021-11-03,2,44,1006.637302,14768.971554,4428.732729,34169.477,3062.021995
3,2021-11-04,3,44,1007.108948,14793.617235,4452.543789,34432.043,3059.876766
4,2021-11-05,4,44,1007.500128,14834.681366,4457.262093,34424.100,3058.086760


## 8가지 특성으로 종목별 종가 예측

In [71]:
stock_pred_dict = {}
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])
for code in tqdm(stock_list['종목코드'].values):
    code_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    code_data = pd.merge(Business_days, code_data, how = 'outer')
    train['Close'] = code_data['Close']

    model=setup(train, target = 'Close', silent=True, fold=3, fold_shuffle=True)
    tuned_model = tune_model(compare_models(sort='MAPE', n_select=1))
    tuned_model_ = finalize_model(tuned_model)
    pred = predict_model(tuned_model_, test)
    stock_pred_dict[code] = pred['Label']

  0%|          | 0/370 [00:00<?, ?it/s]

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:11:31
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies


  0%|          | 0/370 [00:00<?, ?it/s]


KeyError: 'Close'

In [ ]:
len(stock_pred_dict)

3

In [ ]:
stock_pred = pd.DataFrame(stock_pred_dict)
stock_pred

In [ ]:
stock_pred.columns

In [ ]:
stock_pred * 2

In [ ]:
type(stock_pred['005930'])

In [ ]:
final_stock_pred = pd.concat([stock_pred, stock_pred])
final_stock_pred

In [ ]:
final_stock_pred = final_stock_pred.reset_index()
final_stock_pred

In [ ]:
final_stock_pred = final_stock_pred.drop('index',axis=1)
final_stock_pred

In [ ]:
final_stock_pred.columns

# 제출 파일 만들기

In [ ]:
sample_name = 'sample_submission.csv'
sub = pd.read_csv(os.path.join(path,sample_name))
sub

In [ ]:
for code in final_stock_pred.columns:
    sub[code] = final_stock_pred[code]

In [ ]:
sub

In [ ]:
sub.to_csv('sub01_cat01.csv', index=False)